In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.utils import shuffle

from google.colab import files

from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with zipfile.ZipFile('gdrive/My Drive/sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [0]:
df = pd.read_csv('/content/dataset/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", header = None)

In [0]:
df = shuffle(df)

In [0]:
dataset = df.values.astype('str')

In [0]:
len(dataset)

1600000

In [0]:
lables = dataset[:,:1]
tweets = dataset[:,5:]

In [0]:
len(tweets)

1600000

In [0]:
split_tweets = []

for i in tqdm(range(len(tweets))):
  split_tweets.append(tweets[i][0].lower().split(' '))

100%|██████████| 1600000/1600000 [00:10<00:00, 151965.08it/s]


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(split_tweets)

In [0]:
x = tokenizer.texts_to_sequences(split_tweets)

In [0]:
x = pad_sequences(x,25)

In [0]:
del tweets
del split_tweets
del dataset

In [0]:
x = np.array(x)
y = np.array(lables).astype('int')
y = y/2.
y = to_categorical(y, num_classes = 3)

In [0]:
train_x = x[:1550000,:]
train_y = y[:1550000,:]

test_x  = x[1550000:,:]
test_y  = y[1550000:,:]

In [0]:
del lables
del x
del y

In [0]:
model = Sequential()

model.add(Embedding(len(tokenizer.word_counts)+1, 25, input_length = 25))

model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))

model.add(Dropout(0.5))

model.add(Dense(50, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
model.fit(train_x,train_y, validation_data = (test_x,test_y), batch_size = 512, epochs = 2, verbose = 1)

Train on 1550000 samples, validate on 50000 samples
Epoch 1/2
1550000/1550000 [==============================] - 785s 506us/step - loss: 0.4278 - acc: 0.8013 - val_loss: 0.3689 - val_acc: 0.8371
Epoch 2/2
 350208/1550000 [=====>........................] - ETA: 9:59 - loss: 0.2832 - acc: 0.8828